# LSTM Model for Reddit, Twitter, and a combined dataset

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from nltk.stem.porter import *
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import string, csv
import numpy as np
import pandas as pd
%run data_proc.ipynb

## Importing Datasets

In [18]:
# Reddit dataset
REDDIT_TRAIN= '../data/reddit/train-balanced-sarcasm.csv'

# Twitter dataset
TWITTER_TRAIN = '../data/Twitter/train/Train_v1.txt'
EMOJI_DICT = '../data/Twitter/emoji_unicode_names_final.txt'
WORD_LIST = '../data/Twitter/word_list.txt'
WORD_SPLIT = '../data/Twitter/word_split.txt'

TypeError: loaddata() missing 3 required positional arguments: 'word_file_path', 'split_word_path', and 'emoji_file_path'

In [ ]:
twitter_df = loaddata(twitter_train)
with open(twitter_train, 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    df = pd.DataFrame(data)

In [15]:
X, y = [], []
punct = string.punctuation
stemmer = PorterStemmer() 

# Lowercase_sentences
# for d in data[1:]: 
#     label = d[0]
#     text = d[1]
#     text = text.lower()
    
    #Remove punctuation
#     text = [c for c in text if not (c in punct)]    
#     text = ' '.join(text)    
    
    #Stemming and removing stop words gave worse accuracy
    
    #Remove stop words
    #stops = set(stopwords.words("english"))
    #text = [w for w in text if not w in stops and len(w) >= 3]
 
    #Stem the words
    #text = [stemmer.stem(w) for w in text]
#     X.append(text)
#     Y.append(int(label))
#The model automatically splits train and validation
# X_train = X[:700000]
# Y_Train = Y[:700000]
# X_Test = X[700000:900000]
# Y_Test = Y[700000:900000]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)
# print(len(X_train),len(X_test),len(y_train),len(y_test))
print(df.head)

<bound method NDFrame.head of                                                        0  \
0      TrainSen\t0\t@0430yes i hope youre lurking rn....   
1      TrainSen\t0\t05 really taught me a valuable le...   
2      TrainSen\t0\t@098BERRY Never had a voice to pr...   
3      TrainSen\t0\t@0hMySt4rs Rest in peace & love t...   
4      TrainSen\t0\t100 days until Christmas! 🌲 #too ...   
5      TrainSen\t0\t@100_ThingsILove @WhatNinaSpotted...   
6      TrainSen\t0\t100 words short of the word requi...   
7      TrainSen\t0\t@1010xlhacker it was nice hanging...   
8      TrainSen\t0\t10k walk this morning. We did an ...   
9      TrainSen\t0\t10 minutes to eat #challengeaccepted   
10     TrainSen\t0\t10 Times One Direction Had the Pe...   
11     TrainSen\t0\t#10TurnOns Yet again if you list ...   
12     TrainSen\t0\t@11aawwgg021 wow! sweet butt for ...   
13           TrainSen\t0\t11 o'clock cant come no faster   
14     TrainSen\t0\t11pm on a Saturday night and Im i...   
15        

In [3]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
dat = pad_sequences(sequences, maxlen=50)

In [4]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.fit(dat, np.array(Y_Train), validation_split=0.4, epochs=3)
model.save('LSTM_model.h5')

Train on 0 samples, validate on 0 samples
Epoch 1/3


AttributeError: 'ProgbarLogger' object has no attribute 'log_values'

In [ ]:
#Test the model
sent = input("Give me a sentence:")
sent = sent.lower()

model.load_weights('Reddit_LSTM.h5')
out = model.predict(sent)
if sum(out)/len(out):
    print("Your sentence is sarcastic")
else:
    print("Your sentence is not sarcastic")
predict = model.predict(pad_sequences(tokenizer.texts_to_sequences(X_Test),maxlen=50))
correct = 0
output = []
for i in predict:
    if i<0.50:
        output.append(0)
    else:
        output.append(1)
        
print(output[:10])
print(Y_Test[:10])
for i in range(0,len(Y_Test)):
    if output[i] == Y_Test[i]:
        correct+=1
        
print("Test Accuracy: "+ str(correct/len(Y_Test)))